In [ ]:
# Import the required libraries
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.


url = 'https://www.funda.nl/detail/koop/verkocht/amsterdam/appartement-amsteldijk-143-3/43425080/'

In [ ]:
# Make a get request to retrieve page
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})

# Resolve the response
if response.status_code==200:
    html_content = response.text.encode('utf-8')
else:
    print(response.status_code)


# Parse HTML content
# Parse HTML page with BeautifulSoup
soup = BeautifulSoup(html_content)

In [ ]:
headless = soup.find_all('div',   {"id" :  re.compile('headlessui-disclosure-panel.*')})

In [ ]:
from openai import OpenAI
import os

print(os.environ.items())
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [ ]:
content="\n".join(list(map(lambda x: x.get_text(), headless)))
prompt = f"""
You are given the information about a dutch house listing and your task is to generate a summary of the listing.

Please format the output in Markdownv2 so that it can be used to send a message from a telegram bot to a house hunter. Make sure that there is a tl;dr execute summary at the beginning with the most important information like size, number of bedrooms, highlights and location! Only include the most important information and discard everything else. The message should be maximum 100 words long with the most important information at the beginning.

Do not wrap the text in a markdown code block, just use the markdown syntax for formatting.

Content:
\"\"\"
{content}
\"\"\"
"""

completion=client.chat.completions.create(messages=[{"role":"user","content":prompt}], model="gpt-4o")

In [ ]:
message=completion.choices[0].message.content
print(message)

In [ ]:
from telegram_bot import TelegramBot
import asyncio
telegram_token = os.environ["TELEGRAM_TOKEN"]
chat_id = os.environ["CHAT_ID"]
telegramBot = TelegramBot(chat_id, telegram_token)
import nest_asyncio
nest_asyncio.apply()

c = message.replace("**","*").replace(".", "\\.").replace(">", "\\>").replace("<", "\\<").replace("_","\\_").replace("-","\\-").replace("(","\\(").replace(")","\\)").replace("k\\.k\\.", "")
print(c)
telegramBot.postMessage(c)
